### 4.6: Merging and Exporting Data - Part 2 
### Contents
- Merging Instacart Data
- Exporting Data in Pickle Format
- Task

In [1]:
# Import libraries

import pandas as pd
import numpy as np
import os

In [2]:
# Set display options for better viewing

pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 100)  # Limit columns
pd.set_option('display.max_rows', 50)      # Limit rows

#### **Merging Instacart Data**

In [3]:
# Create shortcut for data file
path= r'/Users/anjanpakhrin/Documents/Instacart Basket Analysis'

In [4]:
# Create path to data files
df_ords_prior = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'orders_products_prior.csv'), index_col = False)
df_ords = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_checked.csv'), index_col = False)
df_prods = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'products_checked.csv'), index_col = False)

In [5]:
df_prods.shape

(49672, 5)

#### - Checking Output

In [6]:
# Check the output
df_ords_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [7]:
# Check the output of up-to-date version of "orders data"
df_ords.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,first_order
0,2539329,1,1,2,8,NaN,True
1,2398795,1,2,3,7,15.0,False
2,473747,1,3,3,12,21.0,False
3,2254736,1,4,4,7,29.0,False
4,431534,1,5,4,15,28.0,False


### Consistency checks for "**df_ords_prior**"

#### - Checking for mixed data types

In [8]:
# Check for mixed data types
for col in df_ords_prior.columns.tolist():
    weird = (df_ords_prior[[col]].map(type) != df_ords_prior[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_ords_prior[weird]) > 0:
             print('Data type:',(col))

- No mixed data type found

#### - Finding missing values in "**df_ords_prior**"

In [9]:
# Check missing values
df_ords_prior.isnull().sum()

order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64

- No missing value

#### - Finding Duplicates

In [10]:
# Check Duplicates
df_dups = df_ords_prior[df_ords_prior.duplicated()]
if df_dups.empty:
    print("No duplicates found!")
else:
    print(f"Found{len(df_dups)} duplicates row(s):")

No duplicates found!


#### - Checking Dimensions of Dataframes

In [11]:
# Check dimension of "df_ords_priro"
df_ords_prior.shape

(32434489, 4)

In [12]:
# Check dimensions of "df_ords"
df_ords.shape

(3421083, 7)

#### - Merging "df_ords" and "df_ords_prior" (common key: **order_id**)

In [13]:
# Create new merged dataframe "df_merged_large"; Check match
df_merged_large = df_ords.merge(df_ords_prior, on = 'order_id', indicator = True)

In [14]:
# Check output (first 5 rows)
df_merged_large.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,_merge
0,2539329,1,1,2,8,NaN,True,196,1,0,both
1,2539329,1,1,2,8,NaN,True,14084,2,0,both
2,2539329,1,1,2,8,NaN,True,12427,3,0,both
3,2539329,1,1,2,8,NaN,True,26088,4,0,both
4,2539329,1,1,2,8,NaN,True,26405,5,0,both


In [15]:
# Check dimensions of merged dataframe
df_merged_large.shape

(32434489, 11)

In [16]:
# Check matching criterion
df_merged_large['_merge'].value_counts()

_merge
both          32434489
left_only            0
right_only           0
Name: count, dtype: int64

In [17]:
# Merge dataframe using "outer join"
df_merged_large_outer = df_ords.merge(df_ords_prior, on = 'order_id', indicator = True, how = 'outer')

In [18]:
df_merged_large_outer['_merge'].value_counts()

_merge
both          32434489
left_only       206209
right_only           0
Name: count, dtype: int64

##### *No full match between 2 dataframes*

#### **Exporting Data in Pickle Format**

#### - Dropping flagging column "_merge" for exporting
--> no more necessary for further analysis

In [19]:
# Drop the "_merge" column
df_merged_large = df_merged_large.drop(columns=['_merge'])

In [20]:
df_merged_large.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 10 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   order_number            int64  
 3   order_day_of_week       int64  
 4   order_hour_of_day       int64  
 5   days_since_prior_order  float64
 6   first_order             bool   
 7   product_id              int64  
 8   add_to_cart_order       int64  
 9   reordered               int64  
dtypes: bool(1), float64(1), int64(8)
memory usage: 2.2 GB


In [21]:
# Changing data types to reduce file size
df_merged_large['order_id'] = df_merged_large['order_id'].astype('int32')  
df_merged_large['user_id'] = df_merged_large['user_id'].astype('int32')  
df_merged_large['order_number'] = df_merged_large['order_number'].astype('int32')
df_merged_large['order_day_of_week'] = df_merged_large['order_day_of_week'].astype('int32')  
df_merged_large['order_hour_of_day'] = df_merged_large['order_hour_of_day'].astype('int32') 
df_merged_large['days_since_prior_order']  = df_merged_large['days_since_prior_order'].astype('float32')
df_merged_large['product_id'] = df_merged_large['product_id'].astype('int32')  
df_merged_large['add_to_cart_order'] = df_merged_large['add_to_cart_order'].astype('int32')  
df_merged_large['reordered'] = df_merged_large['reordered'].astype('int32')

In [22]:
# Export data to csv
df_merged_large.to_csv(os.path.join(path, '02 Data','Prepared Data', 'orders_products_combined.csv'))

In [23]:
df_merged_large.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 10 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int32  
 1   user_id                 int32  
 2   order_number            int32  
 3   order_day_of_week       int32  
 4   order_hour_of_day       int32  
 5   days_since_prior_order  float32
 6   first_order             bool   
 7   product_id              int32  
 8   add_to_cart_order       int32  
 9   reordered               int32  
dtypes: bool(1), float32(1), int32(8)
memory usage: 1.1 GB


### **Task 4.6**

#### **Step 2: Export merged file in pickle format "orders_products_combined"**

In [24]:
# Export data to pkl
df_merged_large.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_combined.pkl'))

In [25]:
# Check shape
df_merged_large.shape

(32434489, 10)

#### **Step 3: Import "*orders_products_combined*"**

In [26]:
# Import combined dataframe (path to the folder reated above)
df_orders_products_combined = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_combined.pkl'))

In [27]:
# Recall first 5 rows to check
df_orders_products_combined.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered
0,2539329,1,1,2,8,NaN,True,196,1,0
1,2539329,1,1,2,8,NaN,True,14084,2,0
2,2539329,1,1,2,8,NaN,True,12427,3,0
3,2539329,1,1,2,8,NaN,True,26088,4,0
4,2539329,1,1,2,8,NaN,True,26405,5,0


#### **Step 4: Check the shape of "*orders_products_combined*"**

In [28]:
# Check shape
df_orders_products_combined.shape

(32434489, 10)

In [29]:
# Compare shape of "df_merged_large" and "orders_products_combined"
print('Shape of df_merged_large:', df_merged_large.shape)
print('Shape of df_orders_products_combined:', df_orders_products_combined.shape)

# Check if they are identical
if df_merged_large.shape == df_orders_products_combined.shape:
    print('DataFrames are IDENTICAL.')
else:
    print('DataFrames are DIFFERENT.')

Shape of df_merged_large: (32434489, 10)
Shape of df_orders_products_combined: (32434489, 10)
DataFrames are IDENTICAL.


#### **Step 5: Combining *"orders_products_combined"* and wrangled, cleaned, and deduped dataframe of *"products"* data set**

#### - Checking shape of DataFrames

In [30]:
# Checking shape of "orders_products_combined" and "df_prods" (products)
print('Shape of df_orders_products_combined:', df_orders_products_combined.shape)
print('Shape of df_prods:', df_prods.shape)

Shape of df_orders_products_combined: (32434489, 10)
Shape of df_prods: (49672, 5)


In [31]:
df_prods.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49672 entries, 0 to 49671
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_id     49672 non-null  int64  
 1   product_name   49672 non-null  object 
 2   aisle_id       49672 non-null  int64  
 3   department_id  49672 non-null  int64  
 4   prices         49672 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 1.9+ MB


In [32]:
# Changing data type of_prods to match the data types of df_orders_products_combined
df_prods['product_id'] = df_prods['product_id'].astype('int32')
df_prods['aisle_id'] = df_prods['aisle_id'].astype('int32')
df_prods['department_id'] = df_prods['department_id'].astype('int32')
df_prods['prices'] = df_prods['prices'].astype('float32')

In [33]:
# Confirming changes
df_prods.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49672 entries, 0 to 49671
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_id     49672 non-null  int32  
 1   product_name   49672 non-null  object 
 2   aisle_id       49672 non-null  int32  
 3   department_id  49672 non-null  int32  
 4   prices         49672 non-null  float32
dtypes: float32(1), int32(3), object(1)
memory usage: 1.1+ MB


#### - Checking for common key columns

In [34]:
# Check output of products data set
df_prods.head()

,product_id,product_name,aisle_id,department_id,prices
0,1,Chocolate Sandwich Cookies,61,19,5.8
1,2,All-Seasons Salt,104,13,9.3
2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,5,Green Chile Anytime Sauce,5,13,4.3


In [35]:
# Check output for combined data set
df_orders_products_combined.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered
0,2539329,1,1,2,8,NaN,True,196,1,0
1,2539329,1,1,2,8,NaN,True,14084,2,0
2,2539329,1,1,2,8,NaN,True,12427,3,0
3,2539329,1,1,2,8,NaN,True,26088,4,0
4,2539329,1,1,2,8,NaN,True,26405,5,0


#### - Identifying "common columns"

In [36]:
# Find common columns
common_columns = list(set(df_prods.columns) & set(df_orders_products_combined.columns))
print('Common column/s between both DataFrames:', common_columns)

Common column/s between both DataFrames: ['product_id']


In [37]:
# Checking data type of product_id in df_orders_products_combined
df_orders_products_combined['product_id'].dtype

dtype('int32')

In [38]:
# Checking data type of product_id in df_prods
df_prods['product_id'].dtype

dtype('int32')

#### - Merging DataFrames with default JOIN with common key "product_id"

In [39]:
# Create new merged dataframe "df_merged_large"; Check match
df_prods_merged_large_inner = df_orders_products_combined.merge(df_prods, on = 'product_id', indicator = True)

In [40]:
# Check output
df_prods_merged_large_inner.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77,7,9.0,both
1,2539329,1,1,2,8,NaN,True,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,both
2,2539329,1,1,2,8,NaN,True,12427,3,0,Original Beef Jerky,23,19,4.4,both
3,2539329,1,1,2,8,NaN,True,26088,4,0,Aged White Cheddar Popcorn,23,19,4.7,both
4,2539329,1,1,2,8,NaN,True,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,both


In [41]:
# Check matching criterion - checking merge flag frequency
df_prods_merged_large_inner['_merge'].value_counts()

_merge
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64

#### **PERFECT MATCH:** Common key "product_id" exists in both DataFrames, which is ideal outcome, data intergrity is perfect. Let's verify this result with another JOIN type as follows.

#### **Step 6: Confirm the results using merge flag**

In [42]:
# Merge dataframe using "outer join"
df_prods_merged_large_outer = df_orders_products_combined.merge(df_prods, on = 'product_id', how = 'outer', indicator = True)

In [43]:
# Check matching criterion - checking merge flag frequency
df_prods_merged_large_outer['_merge'].value_counts()

_merge
both          32404859
left_only        30200
right_only          11
Name: count, dtype: int64

**SOME UNMATCHED RECORDS:** We actually don't have a full match. There are 30,200 **product_id** which does not exist in df_prods. It verifies that the result from step 6 is correct, where the result from step 5 is not correct.

In [44]:
# Check dimensions of merged dataframe with "OUTER JOIN" for checking
df_prods_merged_large_outer.shape

(32435070, 15)

#### Step 7: Export new merged DataFrame as **"ords_prods_merge"**

1. Drop flagging column "_merge" for exporting¶
--> not required for analysis

In [45]:
# Drop the "_merge" column
df_prods_merged_large_inner = df_prods_merged_large_inner.drop(columns=['_merge'])

In [46]:
df_prods_merged_large_inner.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77,7,9.0
1,2539329,1,1,2,8,NaN,True,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5
2,2539329,1,1,2,8,NaN,True,12427,3,0,Original Beef Jerky,23,19,4.4
3,2539329,1,1,2,8,NaN,True,26088,4,0,Aged White Cheddar Popcorn,23,19,4.7
4,2539329,1,1,2,8,NaN,True,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,1.0


In [47]:
# Export data to pkl
df_prods_merged_large_inner.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_merge.pkl'))

In [48]:
# Export data to csv
df_prods_merged_large_inner.to_csv(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_merge.csv'))

In [49]:
# Check shape of ddf_prods_merged_large_inner
df_prods_merged_large_inner.shape

(32404859, 14)